In [ ]:
# from ssi_fc_data import *
import config
import requests 
import pandas as pd
import psycopg2
from sqlalchemy import create_engine


In [ ]:
# Connect to database
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5432"  # default 5432
)

# Add cursor
cur = conn.cursor()

In [ ]:
from datetime import datetime
current_time = datetime.now()
print("Current Time:", current_time)

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [ ]:


url = "https://fc-data.ssi.com.vn/api/v2/Market/AccessToken"

payload = {
    "consumerID": config.consumerID,
    "consumerSecret": config.consumerSecret
}

headers = {
    "Content-Type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)

if response.status_code == 200:
    access_token = response.json()['data'].get("accessToken")

In [ ]:
# Replace with the token you obtained
url = "https://fc-data.ssi.com.vn/api/v2/Market/Securities"

headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

params = {
    "market": "HOSE",
    "pageIndex": 1,
    "pageSize": 1000
}

# Send the GET request
response = requests.get(url, headers=headers,params=params)

# Print response
print("Response Status:", response.status_code)
print("Response Body:", response.json()['data'])

In [ ]:
# Extract data from the response JSON
data = response.json().get('data', [])

# Convert the data into a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
display(df)

In [ ]:
# Replace with the token you obtained
url = "https://fc-data.ssi.com.vn/api/v2/Market/SecuritiesDetails"

headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

params = {
    "market": "HOSE",
    "pageIndex": 1,
    "pageSize": 1000
    # "symbol": "FPT"
}

# Send the GET request
response = requests.get(url, headers=headers,params=params)

# Print response
# print("Response Status:", response.status_code)
# print("Response Body:", response.json()['data'])

# Extract data from the response JSON
data = response.json()['data'][0]['RepeatedInfo']

# Convert the data into a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
display(df)




In [ ]:
# Replace with the token you obtained
url = " https://fc-data.ssi.com.vn/api/v2/Market/DailyStockPrice"

headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

params = {
    "symbol": "",
    "fromDate":"16/04/2025",
    "toDate": "16/04/2025",
    "pageIndex": 1,
    "pageSize": 100,
    "market": "HOSE"
}

# Send the GET request
response = requests.get(url, headers=headers,params=params)

# Print response
print("Response Status:", response.status_code)
print("Response Body:", response.json()['data'])

# Extract data from the response JSON
data = response.json().get('data', [])

# Convert the data into a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
display(df)

In [ ]:
# Replace with the token you obtained
url = "https://fc-data.ssi.com.vn/api/v2/Market/DailyIndex"

headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

params = {
    "indexId": "VNIndex",
    "fromDate":"01/04/2025",
    "toDate": "16/04/2025",
    "pageIndex": 1,
    "pageSize": 1000,
    "ascending": False,
}

# Send the GET request
response = requests.get(url, headers=headers,params=params)

# Print response
print("Response Status:", response.status_code)
print("Response Body:", response.json())

# Extract data from the response JSON
data = response.json().get('data', [])

# Convert the data into a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
display(df)


In [ ]:
# Replace with the token you obtained
url = "https://fc-data.ssi.com.vn/api/v2/Market/IntradayOhlc"

headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

params = {
    "symbol": "SSI",
    "fromDate":"11/04/2025",
    "toDate": "11/04/2025",
    "pageIndex": 1,
    "pageSize": 1000,
    "ascending": False,
}

# Send the GET request
response = requests.get(url, headers=headers,params=params)

# Print response
print("Response Status:", response.status_code)
print("Response Body:", response.json())

# Extract data from the response JSON
data = response.json().get('data', [])

# Convert the data into a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
display(df)
